# Airflow

This section demonstrates the use of [Apache Airflow](https://airflow.apache.org/) as backend.

Start by running a local Airflow instance with some generated test DAGs:

```commandline
cd s2gos-airflow
pixi install
pixi run airflow standalone
```

Then run the S2GOS gateway server with the local Airflow instance (assuming
the local Airflow webserver runs on http://localhost:8080):

```commandline
pixi shell
s2gos-server run -- wraptile.services.airflow:service --airflow-password=a8e7f4bb230
```

Get the airflow user password from `s2gos-airflow/.airflow/simple_auth_manager_passwords.json.generated`.

In [1]:
from s2gos_client import Client
from s2gos_common.models import ProcessRequest

In [2]:
client = Client(server_url="http://127.0.0.1:8008")
client

In [3]:
client.get_capabilities()

Capabilities(title='Airflow Service', description='A gateway API compliant with OGC API - Processes that uses an Airflow backend.', links=[Link(href='http://127.0.0.1:8008/', rel='self', type='application/json', hreflang='en', title='get_capabilities'), Link(href='http://127.0.0.1:8008/openapi.json', rel='service', type='application/json', hreflang='en', title='openapi'), Link(href='http://127.0.0.1:8008/docs', rel='service', type='text/html', hreflang='en', title='swagger_ui_html'), Link(href='http://127.0.0.1:8008/docs/oauth2-redirect', rel='service', type='text/html', hreflang='en', title='swagger_ui_redirect'), Link(href='http://127.0.0.1:8008/redoc', rel='service', type='text/html', hreflang='en', title='redoc_html'), Link(href='http://127.0.0.1:8008/', rel='service', type='application/json', hreflang='en', title='get_capabilities'), Link(href='http://127.0.0.1:8008/conformance', rel='service', type='application/json', hreflang='en', title='get_conformance'), Link(href='http://127

In [4]:
client.get_conformance()

ConformanceDeclaration(conformsTo=['http://www.opengis.net/spec/ogcapi-processes-1/1.0/conf/core', 'http://www.opengis.net/spec/ogcapi-processes-1/1.0/conf/ogc-process-description', 'http://www.opengis.net/spec/ogcapi-processes-1/1.0/conf/json', 'http://www.opengis.net/spec/ogcapi-processes-1/1.0/conf/oas30', 'http://www.opengis.net/spec/ogcapi-processes-1/1.0/conf/job-list', 'http://www.opengis.net/spec/ogcapi-processes-1/1.0/conf/dismiss'])

In [5]:
client.get_processes()

ProcessList(processes=[ProcessSummary(title='Prime Processor', description='Returns the list of prime numbers between a `min_val` and `max_val`. ', keywords=None, metadata=None, additionalParameters=None, id='primes_between', version='0.0.0', jobControlOptions=None, outputTransmission=None, links=None), ProcessSummary(title='BaseModel Test', description=None, keywords=None, metadata=None, additionalParameters=None, id='return_base_model', version='0.0.0', jobControlOptions=None, outputTransmission=None, links=None), ProcessSummary(title='Generate scene for testing', description='Simulate a set scene images slices for testing. Creates an xarray dataset with `periodicity` time slices and writes it as Zarr into a temporary location. Requires installed `dask`, `xarray`, and `zarr` packages.', keywords=None, metadata=None, additionalParameters=None, id='simulate_scene', version='0.0.0', jobControlOptions=None, outputTransmission=None, links=None), ProcessSummary(title='Sleep Processor', des

In [6]:
client.get_process(process_id="sleep_a_while")

ProcessDescription(title='Sleep Processor', description='Sleeps for `duration` seconds. Fails on purpose if `fail` is `True`. Returns the effective amount of sleep in seconds.', keywords=None, metadata=None, additionalParameters=None, id='sleep_a_while', version='0.0.0', jobControlOptions=None, outputTransmission=None, links=None, inputs={}, outputs={})

In [7]:
client.get_jobs()

JobList(jobs=[JobInfo(processID='primes_between', type=<JobType.process: 'process'>, jobID='manual__2025-07-15T11:12:09.473432+00:00', status=<JobStatus.successful: 'successful'>, message=None, created=datetime.datetime(2025, 7, 15, 11, 12, 9, 501829, tzinfo=TzInfo(UTC)), started=datetime.datetime(2025, 7, 15, 11, 12, 9, 689505, tzinfo=TzInfo(UTC)), finished=datetime.datetime(2025, 7, 15, 11, 12, 13, 599646, tzinfo=TzInfo(UTC)), updated=datetime.datetime(2025, 7, 15, 11, 12, 13, 592815, tzinfo=TzInfo(UTC)), progress=None, links=None, traceback=None), JobInfo(processID='primes_between', type=<JobType.process: 'process'>, jobID='primes_between__20250909133119_1', status=<JobStatus.successful: 'successful'>, message=None, created=datetime.datetime(2025, 9, 9, 13, 31, 19, 770932, tzinfo=TzInfo(UTC)), started=datetime.datetime(2025, 9, 9, 13, 31, 20, 451716, tzinfo=TzInfo(UTC)), finished=datetime.datetime(2025, 9, 9, 13, 31, 24, 8108, tzinfo=TzInfo(UTC)), updated=datetime.datetime(2025, 9, 

In [8]:
client.execute_process(process_id="primes_between", request=ProcessRequest())

JobInfo(processID='primes_between', type=<JobType.process: 'process'>, jobID='primes_between__20250909142749_1', status=<JobStatus.accepted: 'accepted'>, message=None, created=datetime.datetime(2025, 9, 9, 14, 27, 49, 355547, tzinfo=TzInfo(UTC)), started=None, finished=None, updated=None, progress=None, links=None, traceback=None)

In [9]:
client.execute_process(process_id="sleep_a_while", request=ProcessRequest())

JobInfo(processID='sleep_a_while', type=<JobType.process: 'process'>, jobID='sleep_a_while__20250909142749_2', status=<JobStatus.accepted: 'accepted'>, message=None, created=datetime.datetime(2025, 9, 9, 14, 27, 49, 520012, tzinfo=TzInfo(UTC)), started=None, finished=None, updated=None, progress=None, links=None, traceback=None)

In [10]:
client.get_jobs()

JobList(jobs=[JobInfo(processID='primes_between', type=<JobType.process: 'process'>, jobID='manual__2025-07-15T11:12:09.473432+00:00', status=<JobStatus.successful: 'successful'>, message=None, created=datetime.datetime(2025, 7, 15, 11, 12, 9, 501829, tzinfo=TzInfo(UTC)), started=datetime.datetime(2025, 7, 15, 11, 12, 9, 689505, tzinfo=TzInfo(UTC)), finished=datetime.datetime(2025, 7, 15, 11, 12, 13, 599646, tzinfo=TzInfo(UTC)), updated=datetime.datetime(2025, 7, 15, 11, 12, 13, 592815, tzinfo=TzInfo(UTC)), progress=None, links=None, traceback=None), JobInfo(processID='primes_between', type=<JobType.process: 'process'>, jobID='primes_between__20250909133119_1', status=<JobStatus.successful: 'successful'>, message=None, created=datetime.datetime(2025, 9, 9, 13, 31, 19, 770932, tzinfo=TzInfo(UTC)), started=datetime.datetime(2025, 9, 9, 13, 31, 20, 451716, tzinfo=TzInfo(UTC)), finished=datetime.datetime(2025, 9, 9, 13, 31, 24, 8108, tzinfo=TzInfo(UTC)), updated=datetime.datetime(2025, 9, 

In [11]:
client.get_job_results("sleep_a_while__20250909140046_2")

JobResults(root={'return_value': InlineOrRefValue(root=InlineValue(root=10.069934606552124))})

In [15]:
client.get_job_results("primes_between__20250909142749_1")

JobResults(root={'return_value': InlineOrRefValue(root=InlineValue(root=[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]))})

In [12]:
# for job in client.get_jobs().jobs:
#     client.dismiss_job(job.jobID)

In [13]:
client.get_jobs()

JobList(jobs=[JobInfo(processID='primes_between', type=<JobType.process: 'process'>, jobID='manual__2025-07-15T11:12:09.473432+00:00', status=<JobStatus.successful: 'successful'>, message=None, created=datetime.datetime(2025, 7, 15, 11, 12, 9, 501829, tzinfo=TzInfo(UTC)), started=datetime.datetime(2025, 7, 15, 11, 12, 9, 689505, tzinfo=TzInfo(UTC)), finished=datetime.datetime(2025, 7, 15, 11, 12, 13, 599646, tzinfo=TzInfo(UTC)), updated=datetime.datetime(2025, 7, 15, 11, 12, 13, 592815, tzinfo=TzInfo(UTC)), progress=None, links=None, traceback=None), JobInfo(processID='primes_between', type=<JobType.process: 'process'>, jobID='primes_between__20250909133119_1', status=<JobStatus.successful: 'successful'>, message=None, created=datetime.datetime(2025, 9, 9, 13, 31, 19, 770932, tzinfo=TzInfo(UTC)), started=datetime.datetime(2025, 9, 9, 13, 31, 20, 451716, tzinfo=TzInfo(UTC)), finished=datetime.datetime(2025, 9, 9, 13, 31, 24, 8108, tzinfo=TzInfo(UTC)), updated=datetime.datetime(2025, 9, 

In [14]:
client.get_job("primes_between__20250909133119_1")

JobInfo(processID='primes_between', type=<JobType.process: 'process'>, jobID='primes_between__20250909133119_1', status=<JobStatus.successful: 'successful'>, message=None, created=datetime.datetime(2025, 9, 9, 13, 31, 19, 770932, tzinfo=TzInfo(UTC)), started=datetime.datetime(2025, 9, 9, 13, 31, 20, 451716, tzinfo=TzInfo(UTC)), finished=datetime.datetime(2025, 9, 9, 13, 31, 24, 8108, tzinfo=TzInfo(UTC)), updated=datetime.datetime(2025, 9, 9, 13, 31, 24, 3845, tzinfo=TzInfo(UTC)), progress=None, links=None, traceback=None)